In [2]:
import os
import psycopg2
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv(".env")

# Obtenir les informations d'identification de la base de données depuis le fichier .env
USER_PSQL = os.environ.get("POSTGRESQL_LOCAL_USER")
PASSWORD_PSQL = os.environ.get("POSTGRESQL_LOCAL_PASSWORD")


def open_connection():
    try:
        conn = psycopg2.connect(
            user=USER_PSQL,
            password=PASSWORD_PSQL,
            host="localhost",
            port="5432",
            dbname="app_auth"
        )
        return conn
    except Exception as e:
        print(f"Erreur de connexion à la base de données: {e}")
        return None


def close_connection(conn):
    if conn is not None:
        conn.close()

## Question 1: Création de la Table `user_for_comparison`

Nous allons créer une table `user_for_comparison` qui a la même structure que la table `user`. Cette table sera utilisée pour comparer les performances des requêtes avant et après l'ajout d'un index.

In [ ]:
conn = open_connection()
cursor = conn.cursor()

create_table_script = """
CREATE TABLE IF NOT EXISTS user_for_comparison (
    user_id SERIAL PRIMARY KEY,
    firstname VARCHAR(255),
    lastname VARCHAR(255),
    email VARCHAR(255) UNIQUE,
    username VARCHAR(255) UNIQUE,
    password VARCHAR(255),
    created_at TIMESTAMP WITHOUT TIME ZONE
);
"""
cursor.execute(create_table_script)
conn.commit()

close_connection(conn)

### Insertion des Données

Nous allons maintenant insérer 100 000 lignes de données générées de manière crédible dans la table `user_for_comparison`.

In [ ]:
from faker import Faker
import random

faker = Faker()
conn = open_connection()
cursor = conn.cursor()

for _ in range(100):
    for _ in range(1000):
        try:
            cursor.execute(
                "INSERT INTO user_for_comparison (firstname, lastname, email, username, password, created_at) VALUES (%s, %s, %s, %s, %s, %s)",
                (
                    faker.first_name(),
                    faker.last_name(),
                    faker.email(),
                    faker.user_name() + str(random.randint(1, 1000000)),
                    faker.password(),
                    faker.date_between(start_date="-2y", end_date="today")
                )
            )
        except psycopg2.errors.UniqueViolation:
            conn.rollback()
        else:
            conn.commit()

close_connection(conn)

### Question 2: Mesure du Temps d'Exécution des Requêtes

Nous allons mesurer le temps d'exécution des requêtes `SELECT` avec un filtre `WHERE` sur différents `username` dans la table `user_for_comparison`.

In [4]:
import time

conn = open_connection()
cursor = conn.cursor()

usernames_to_test = ["example_username1", "example_username2", "example_username3"]

for username in usernames_to_test:
    start_time = time.time()
    cursor.execute("SELECT * FROM user_for_comparison WHERE username = %s", (username,))
    cursor.fetchall()
    end_time = time.time()

    print(f"Temps d'exécution pour {username}: {end_time - start_time} secondes")

close_connection(conn)

Temps d'exécution pour example_username1: 0.0 secondes
Temps d'exécution pour example_username2: 0.0 secondes
Temps d'exécution pour example_username3: 0.0 secondes


### Question 3: Création d'un Index sur la Colonne `username`

Nous allons créer un index sur la colonne `username` de la table `user_for_comparison` et répéter les mesures de temps d'exécution des requêtes pour comparer les performances.

In [3]:
conn = open_connection()
cursor = conn.cursor()

create_index_script = "CREATE INDEX idx_username ON user_for_comparison (username);"
cursor.execute(create_index_script)
conn.commit()

for username in usernames_to_test:
    start_time = time.time()
    cursor.execute("SELECT * FROM user_for_comparison WHERE username = %s", (username,))
    cursor.fetchall()
    end_time = time.time()

    print(f"Temps d'exécution après indexation pour {username}: {end_time - start_time} secondes")

close_connection(conn)

DuplicateTable: ERREUR:  la relation « idx_username » existe déjà
